Download the dataset

In [1]:
%%script false # skip this block
!curl -o datasets/big.txt https://norvig.com/big.txt
!curl -o datasets/tinyshakespeare.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

Couldn't find program: 'false'


Load the dataset

In [2]:
# input_file_path = "datasets/big.txt"
input_file_path = "datasets/tinyshakespeare.txt"

with open(input_file_path, 'r', encoding='utf-8') as f:
    train_text = f.read()

print(f"Loaded dataset with {len(train_text)} characters.")

Loaded dataset with 1115394 characters.


In [3]:
print(train_text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


Isolate unique chars

In [4]:
vocab = sorted(list(set(train_text)))
vocab_size = len(vocab)
print(''.join(vocab))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Make a simple char tokenizeer

In [5]:
# %%script false # skip this block
char_to_int = { char:i for i,char in enumerate(vocab) }
int_to_char = { i:char for i,char in enumerate(vocab) }
encode = lambda str: [char_to_int[char] for char in str]
decode = lambda ints: ''.join([int_to_char[i] for i in ints])

print(decode(encode("hello world!")))

hello world!


In [6]:
%%script false # skip this block
import tiktoken
enc = tiktoken.get_encoding('gpt2')

encode = enc.encode
decode = enc.decode

vocab_size = enc.n_vocab
vocab = [enc.decode([i]) for i in range(vocab_size)]

Couldn't find program: 'false'


In [7]:
import torch
data = torch.tensor(encode(train_text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
# %%script false # skip this block

n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

Split data into training and validation sets based on randomly selected chunks

In [9]:
%%script false # skip this block
import torch, random

newline_token = encode("\n")
space_token = encode(" ")

val_fraction = 0.1
num_chunks   = 12

total_len = data.size(0)
val_len   = int(val_fraction * total_len)
chunk_len = val_len // num_chunks
remainder = val_len % num_chunks

# split the entire dataset into chunks
chunks = []
idx = 0
while idx + chunk_len <= total_len:
    chunks.append(data[idx : idx + chunk_len])
    idx += chunk_len

# the leftover chunk (if any) after slicing out as many `chunk_len` blocks as possible. We always put this leftover into training by default
leftover = data[idx:]

val_indices = sorted(random.sample(range(len(chunks)), num_chunks))

# build the validation data (insert newline_token only between non-consecutive chunks)
val_data = []
prev_idx = -2  # something not adjacent to first pick 
for i in val_indices:
    # if the current chunk is NOT directly after the previous one and val_data already has content,
    # then insert a newline token
    if i != prev_idx + 1 and len(val_data) > 0:
        val_data.append(torch.tensor(newline_token, dtype=torch.int64))
    val_data.append(chunks[i])
    prev_idx = i

# concatenate chosen val chunks
val_data = torch.cat(val_data) if val_data else torch.empty(0, dtype=torch.int64)

# if we have a remainder that is significant, tack data from train onto the end. This is ugly and bad and should be changed
# this ensures total validation tokens == val_len but can also result in a small amount of overlap (up to chunk_length -1 tokens) between val and train
if remainder > 100:
    val_data = torch.cat((val_data, data[-remainder:]))

# everything else is training: the unchosen chunks plus leftover
train_indices = sorted(set(range(len(chunks))) - set(val_indices))
train_data_list = []
prev_idx = -2  # so first chunk won't auto-insert a newline
for i in train_indices:
    # if this chunk is not consecutive to the previous one, insert a newline
    if i != prev_idx + 1 and len(train_data_list) > 0:
        train_data_list.append(torch.tensor(newline_token, dtype=torch.int64))
    train_data_list.append(chunks[i])
    prev_idx = i

train_data_list.append(torch.tensor(newline_token, dtype=torch.int64))
train_data_list.append(leftover)
train_data = torch.cat(train_data_list) if train_data_list else torch.empty(0, dtype=torch.int64)


print("Training data size   :", train_data.size(0))
print("Validation data size :", val_data.size(0))
print("Combinded data size  :", val_data.size(0)+train_data.size(0))
print("Original data size   :", total_len)
print(f"Difference in sizes {val_data.size(0)+train_data.size(0)-total_len}, Expected difference near {num_chunks+1} depending on consective chunk frequency and if remainder was added to train and val data")
print(f"Expected validation/data size of {val_fraction}, actual size of {val_data.size(0)/(val_data.size(0)+train_data.size(0))}")
print(f"Expected train/data size of {1-val_fraction}, actual size of {train_data.size(0)/(val_data.size(0)+train_data.size(0))}")

Couldn't find program: 'false'


In [10]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1].tolist()
    target = [y[t].item()]
    print(f"when input is {decode(context)} the target: {decode(target)}")

when input is F the target: i
when input is Fi the target: r
when input is Fir the target: s
when input is Firs the target: t
when input is First the target:  
when input is First  the target: C
when input is First C the target: i
when input is First Ci the target: t


In [12]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print ("----")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {decode(context.tolist())} the target: {decode([target.item()])}")
    print("----")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is L the target: e
when input is Le the target: t
when input is Let the target: '
when input is Let' the target: s
when input is Let's the target:  
when input is Let's  the target: h
when input is Let's h the target: e
when input is Let's he the target: a
----
when input is f the target: o
when input is fo the target: r
when input is for the target:  
when input is for  the target: t
when input is for t the target: h
when input is for th the target: a
when input is for tha the target: t
when input is for that the target:  
----
when input is n the target: t
when input is nt the targe

Bigram Language Model

In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # get the last logit (B,C)
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramLanguageModel()
logits, loss= model(xb, yb)
print(logits.shape)
print(loss)
print(decode(model.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


Optimizing the model

In [14]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [15]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())



2.5727508068084717


In [16]:
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN t l-t E:
I hisgothers je are!-e!
QLYotouciullle'z


In [17]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [18]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        xbow[b,t] = torch.mean(xprev, 0)

In [19]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True) # Lower triangular matrix whose values are relative proportions of the number of ones in each

xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2, rtol=1e-06, atol=1e-06)

True

In [20]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
wei
xbow3 = wei @ x
torch.allclose(xbow, xbow3, rtol=1e-06, atol=1e-06)

True

self-attention

In [21]:
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [28]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [29]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [30]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))